In [1]:
# Para visualización en notebook
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['font.size'] = 22
from IPython.display import Audio, Markdown, Image
import seaborn as sns

# Números y Datos
import numpy as np
import pandas as pd

# Archivos
import urllib.request
import glob

# Análisis de sonido
import spotipy 

# Machine learning
# importar los paquetes para clustering
from sklearn.preprocessing import StandardScaler

# To work with categorical types
from pandas.api.types import CategoricalDtype

Se cargan los datasets.

In [3]:
metadata = pd.read_csv('data/metadata.csv', index_col='id')
audio_features = pd.read_csv('data/audio_features.csv', index_col='id')
audio_analysis = pd.read_csv('data/audio_analysis.csv', index_col='id')

In [4]:
audio_features.head()

,acousticness,analysis_url,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
id,,,,,,,,,,,,,,,,,
5i2sgqnzaxYpvXmGAB592h,0.957,https://api.spotify.com/v1/audio-analysis/5i2s...,0.524,253933,0.1080,0.001230,0,0.139,-18.361,1,0.0493,127.982,4,https://api.spotify.com/v1/tracks/5i2sgqnzaxYp...,audio_features,spotify:track:5i2sgqnzaxYpvXmGAB592h,0.322
4mGzm5ahHxMX5IhVWbU7cl,0.953,https://api.spotify.com/v1/audio-analysis/4mGz...,0.480,249823,0.0322,0.000004,4,0.098,-23.781,1,0.0604,121.766,4,https://api.spotify.com/v1/tracks/4mGzm5ahHxMX...,audio_features,spotify:track:4mGzm5ahHxMX5IhVWbU7cl,0.172
4DEn3hZzSs9MFwFNDla5Y5,0.989,https://api.spotify.com/v1/audio-analysis/4DEn...,0.394,185827,0.1500,0.908000,5,0.342,-21.133,0,0.0353,97.738,3,https://api.spotify.com/v1/tracks/4DEn3hZzSs9M...,audio_features,spotify:track:4DEn3hZzSs9MFwFNDla5Y5,0.170
5P6vo51dtkBYWXswH1twvK,0.174,https://api.spotify.com/v1/audio-analysis/5P6v...,0.461,645894,0.7730,0.702000,9,0.692,-10.316,0,0.1470,130.109,5,https://api.spotify.com/v1/tracks/5P6vo51dtkBY...,audio_features,spotify:track:5P6vo51dtkBYWXswH1twvK,0.488
29TZQMVLv25CBm3bYn2EIQ,0.436,https://api.spotify.com/v1/audio-analysis/29TZ...,0.452,326400,0.5010,0.900000,10,0.110,-13.149,0,0.0329,111.203,4,https://api.spotify.com/v1/tracks/29TZQMVLv25C...,audio_features,spotify:track:29TZQMVLv25CBm3bYn2EIQ,0.602


In [5]:
audio_analysis.head()

,timbre_mean_0,timbre_mean_1,timbre_mean_2,timbre_mean_3,timbre_mean_4,timbre_mean_5,timbre_mean_6,timbre_mean_7,timbre_mean_8,timbre_mean_9,...,pitch_std_2,pitch_std_3,pitch_std_4,pitch_std_5,pitch_std_6,pitch_std_7,pitch_std_8,pitch_std_9,pitch_std_10,pitch_std_11
id,,,,,,,,,,,,,,,,,,,,,
2NChLGjZ93zqggig6DmoLL,39.573680,-53.160322,33.955933,-13.458602,10.908333,-34.851049,1.713744,0.357444,-9.248460,-3.464805,...,0.314326,0.182297,0.175415,0.315322,0.197047,0.190721,0.152499,0.315809,0.165763,0.112952
3TPucnH2T71lgcmhsTZVMT,37.806990,26.905931,45.066553,-1.566750,19.836837,-10.684136,-37.983923,-10.510028,-5.820575,-1.746866,...,0.293350,0.250039,0.339806,0.311496,0.260636,0.341519,0.176243,0.284082,0.300141,0.236217
5ruWnyGX0r6v8b2JisDHDo,50.307463,65.649677,-17.158870,-1.660690,30.536530,-47.617183,1.200309,1.357931,-2.241743,4.456728,...,0.361503,0.331288,0.333903,0.320905,0.290592,0.337753,0.306255,0.320579,0.347926,0.315364
4ZEVoBaSKdJfpeunWmyQfs,38.453838,-46.715751,-16.638915,-3.838850,34.693517,-27.530044,5.632033,2.938340,7.223357,-0.045737,...,0.211055,0.172432,0.307475,0.226574,0.209167,0.271316,0.221715,0.297126,0.209825,0.216391
66aVNocFLApjTfudNYVEiF,52.072411,98.597172,-16.693923,0.843767,42.616539,-42.602253,-3.609067,11.836476,2.011623,0.462091,...,0.277745,0.307850,0.362859,0.300253,0.295673,0.236315,0.301466,0.244227,0.257769,0.281606
